In [1]:
import os
import sys
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras import backend as K
from keras.layers import Embedding
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

Using TensorFlow backend.


In [2]:
train=pd.read_csv("C:/Users/Pratyush/.spyder-py3/mp/train_dataset.csv") 
test=pd.read_csv("C:/Users/Pratyush/.spyder-py3/mp/test_dataset.csv")

In [3]:
aaa=0
eee=0
iii=0
mmm=0
nnn=0
for i in range(train.shape[0]):
    if train["relation_type"][i]=='advise':
        aaa+=1
    if train["relation_type"][i]=='effect':
        eee+=1
    if train["relation_type"][i]=='int':
        iii+=1
    if train["relation_type"][i]=='mechanism':
        mmm+=1 
    if train["relation_type"][i]=='No interaction':
        nnn+=1 
print(aaa,eee,iii,mmm,nnn)

696 1308 146 1054 18030


In [4]:
aa=0
ee=0
ii=0
mm=0
nn=0
for i in range(test.shape[0]):
    if test["relation_type"][i]=='advise':
        aa+=1
    if test["relation_type"][i]=='effect':
        ee+=1
    if test["relation_type"][i]=='int':
        ii+=1
    if test["relation_type"][i]=='mechanism':
        mm+=1 
    if test["relation_type"][i]=='No interaction':
        nn+=1 
print(aa,ee,ii,mm,nn)

167 283 82 245 3919


In [5]:
def remove_punctuation(string): 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
    return string

In [6]:
label_dict={'advise':[1,0,0,0,0], 'effect':[0,1,0,0,0], 'int':[0,0,1,0,0], 'mechanism':[0,0,0,1,0], "No interaction":[0,0,0,0,1]}

In [7]:
train_data=[]
test_data=[]
train_label=[]
test_label=[]
texts = []
labels = []
labels_index = {'advise':0, 'effect':1, 'int':2, 'mechanism':3, 'No interaction':4}

for i in range(train.shape[0]):
    sentence=remove_punctuation(train["sentence_text"][i])
    texts.append(sentence)
    train_label.append(label_dict[train["relation_type"][i]])
    labels.append(labels_index[train["relation_type"][i]])

for i in range(test.shape[0]):
    sentence=remove_punctuation(test["sentence_text"][i])
    texts.append(sentence)
    test_label.append(label_dict[test["relation_type"][i]])  
    labels.append(labels_index[test["relation_type"][i]])

print('Found %s total sentences.' % len(texts))

Found 25930 total sentences.


In [8]:
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100
filter_sizes = [3,4,5]
num_filters = 512
embedding_dim = 100
drop = 0.5
epochs = 20

In [9]:
tokenizer  = Tokenizer(num_words = MAX_WORDS)
tokenizer.fit_on_texts(texts)
sequences =  tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

unique words : 6663
Shape of data tensor: (25930, 100)
Shape of label tensor: (25930, 5)


In [10]:
nb_validation_samples = len(test)

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [11]:
x_train, y_train = shuffle(x_train, y_train)
x_val, y_val = shuffle(x_val, y_val)

In [12]:
embeddings_index = {}
base_path = "C:/Users/Pratyush/.spyder-py3/mp/"
filename = "glove.6B.100d.txt"
path_to_file = os.path.join(base_path, filename)
f = open(path_to_file , 'r', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [13]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [15]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding = embedding_layer(inputs)

reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=5, activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('output.hdf5', monitor='val_f1_m', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='mse', metrics=['accuracy', precision_m, recall_m, f1_m])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     666400      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 100, 100, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)    

In [17]:
print("Traning Model...")
model.fit(x_train, y_train, epochs=epochs, verbose=1)

Traning Model...
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
21234/21234 [==============================] - 311s 15ms/step - loss: 0.0580 - acc: 0.8442 - precision_m: 0.8474 - recall_m: 0.8386 - f1_m: 0.8429
Epoch 2/20
21234/21234 [==============================] - 298s 14ms/step - loss: 0.0547 - acc: 0.8435 - precision_m: 0.8492 - recall_m: 0.8364 - f1_m: 0.8426
Epoch 3/20
21234/21234 [==============================] - 299s 14ms/step - loss: 0.0523 - acc: 0.8458 - precision_m: 0.8533 - recall_m: 0.8370 - f1_m: 0.8449
Epoch 4/20
21234/21234 [==============================] - 307s 14ms/step - loss: 0.0499 - acc: 0.8498 - precision_m: 0.8579 - recall_m: 0.8402 - f1_m: 0.8488
Epoch 5/20
21234/21234 [==============================] - 295s 14ms/step - loss: 0.0487 - acc: 0.8509 - precision_m: 0.8599 - recall_m: 0.8403 - f1_m: 0.8498
Epoch 6/20
21234/21234 [==============================] - 295s 14ms/step - loss: 0.0476 - acc: 0.8539 - precision_m: 0.8612 - recall_m: 0.8429 - 

In [18]:
print("Evaluating Model on train data...")
model.evaluate(x_train, y_train, verbose=1)

Evaluating Model on train data...
21234/21234 [==============================] - 77s 4ms/step


[0.03637998168739351,
 0.8841951587077329,
 0.8882589737237415,
 0.8770839220118678,
 0.8825402482144069]

In [19]:
print("Evaluating Model on test data...")
model.evaluate(x_val, y_val, verbose=1)

Evaluating Model on test data...
4696/4696 [==============================] - 17s 4ms/step


[0.050972747322952525,
 0.8449744464388494,
 0.8527774716357762,
 0.8336882452136207,
 0.8429483672829259]

In [20]:
def precision(tp,fp):
    return tp/(tp+fp)

def recall(tp,fn):
    return tp/(tp+fn)

def f1score(tp,fp,fn):
    p=precision(tp,fp)
    r=recall(tp,fn)
    return 2*(p*r)/(p+r)

def result(a):
    index=np.argmax(a)
    val=[]
    for i in range(len(a)):
        if index==i:
            val.append(1)
        else:
            val.append(0)
    val = np.array(val).astype(int)
    return val

In [21]:
tp_advise=0
fp_advise=0
fn_advise=0
tp_effect=0
fp_effect=0
fn_effect=0
tp_int=0
fp_int=0
fn_int=0
tp_mechanism=0
fp_mechanism=0
fn_mechanism=0
tp_noint=0
fp_noint=0
fn_noint=0
cc = 0
aa = []
for i in range(len(x_val)):
    abc = []
    abc.append(x_val[i])
    v = model.predict([abc], verbose=1)[0]
    aa.append(v)
    val=result(v)
    yy_val = np.array(list(y_val[i])).astype(int)
    val = list(val)
    yy_val = list(yy_val)
    if val == [0,0,0,0,1]:
        cc+=1
    
    if val==yy_val:
        if val==[1,0,0,0,0]:
            tp_advise+=1
        if val==[0,1,0,0,0]:
            tp_effect+=1 
        if val==[0,0,1,0,0]:
            tp_int+=1
        if val==[0,0,0,1,0]:
            tp_mechanism+=1 
        if val==[0,0,0,0,1]:
            tp_noint+=1 
    else:
        if val==[1,0,0,0,0]:
            fp_advise+=1
        if val==[0,1,0,0,0]:
            fp_effect+=1 
        if val==[0,0,1,0,0]:
            fp_int+=1
        if val==[0,0,0,1,0]:
            fp_mechanism+=1
        if val==[0,0,0,0,1]:
            fp_noint+=1
            
        if yy_val==[1,0,0,0,0]:
            fn_advise+=1
        if yy_val==[0,1,0,0,0]:
            fn_effect+=1  
        if yy_val==[0,0,1,0,0]:
            fn_int+=1
        if yy_val==[0,0,0,1,0]:
            fn_mechanism+=1 
        if yy_val==[0,0,0,0,1]:
            fn_noint+=1

print(cc,aa)

1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 7ms/step


1/1 [==============================] - 0s 7ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 6ms/step


...

1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 5ms/step


1/1 [==============================] - 0s 6ms/step


1/1 [==============================] - 0s 8ms/step
4449 [array([7.5335021e-04, 8.3205407e-04, 2.8640975e-03, 4.2105750e-03,
       9.9133986e-01], dtype=float32), array([0.0361156 , 0.01338712, 0.00148379, 0.03045551, 0.918558  ],
      dtype=float32), array([0.00531447, 0.02077642, 0.00280467, 0.00754067, 0.9635638 ],
      dtype=float32), array([0.00143015, 0.08218048, 0.00165125, 0.01095153, 0.90378654],
      dtype=float32), array([0.00851305, 0.06309568, 0.00933063, 0.16573977, 0.7533208 ],
      dtype=float32), array([0.01956099, 0.05720091, 0.0036394 , 0.03154229, 0.88805646],
      dtype=float32), array([0.00271683, 0.07820505, 0.00184829, 0.01196724, 0.90526265],
      dtype=float32), array([0.00271683, 0.07820505, 0.00184829, 0.01196724, 0.90526265],
      dtype=float32), array([0.02972899, 0.04209834, 0.00396612, 0.15182166, 0.7723849 ],
      dtype=float32), array([0.00347229, 0.01105552, 0.00335844, 0.0288701 , 0.9532437 ],
      dtype=float32), array([0.04025557, 0.187860

In [22]:
tp_advise,fp_advise,fn_advise,tp_effect,fp_effect,fn_effect,tp_int,fp_int,fn_int,tp_mechanism,fp_mechanism,fn_mechanism

(40, 20, 127, 84, 64, 199, 0, 1, 82, 21, 17, 224)

In [23]:
tp_noint,fp_noint,fn_noint

(3823, 626, 96)

In [24]:
print(precision(tp_advise,fp_advise))
print(recall(tp_advise,fn_advise))
print(f1score(tp_advise,fp_advise,fn_advise))

0.6666666666666666
0.23952095808383234
0.3524229074889868


In [25]:
print(precision(tp_effect,fp_effect))
print(recall(tp_effect,fn_effect))
print(f1score(tp_effect,fp_effect,fn_effect))

0.5675675675675675
0.2968197879858657
0.38979118329466356


In [26]:
print(precision(tp_int,fp_int))
print(recall(tp_int,fn_int))
print(f1score(tp_int,fp_int,fn_int))

0.0
0.0


ZeroDivisionError: float division by zero

In [27]:
print(precision(tp_mechanism,fp_mechanism))
print(recall(tp_mechanism,fn_mechanism))
print(f1score(tp_mechanism,fp_mechanism,fn_mechanism))

0.5526315789473685
0.08571428571428572
0.14840989399293286


In [28]:
print(precision(tp_noint,fp_noint))
print(recall(tp_noint,fn_noint))
print(f1score(tp_noint,fp_noint,fn_noint))

0.8592942234209935
0.9755039550905843
0.9137189292543022
